1. definir o navegadora a utilizar 
2. importar modulos e bibliotecas
3. entender as requisições na internet
4. conheçer e mapear o processo de coleta de dados dos ETF
5. achar os elementos HTML do site
6. ler a tabela de dados
7. contruir a tabela final

In [287]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [288]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

url = "https://www.etf.com/etfanalytics/etf-finder"

driver.get(url)

"<b>arguments[0]</b>" é uma referência ao primeiro argumento passado para o método <b>execute_script</b> do objeto <b>driver</b> no código.

No caso específico, <b>arguments[0].click()</b> é um comando JavaScript que clica em um elemento HTML. Este elemento é passado como o primeiro argumento da função, ou seja, <b>botao_100</b> é o elemento que será clicado.

A função <b>execute_script</b> permite a execução de código JavaScript no contexto do navegador controlado pelo objeto <b>driver</b>. Ao passar "<b>arguments[0].click();</b>" como primeiro argumento, e <b>botao_100</b> como segundo argumento, estamos indicando que o método click() deve ser invocado no elemento <b>botao_100</b>.

Em resumo, este código clica no elemento <b>botao_100</b> usando a biblioteca de automação de testes <b>driver</b>.

In [289]:
#achar os elementos necessarios do html (copiei o xpath do site no botão +100)

time.sleep(5)

botao_100 = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span''')

driver.execute_script("arguments[0].click();", botao_100)

In [290]:
numeroPaginas = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/label[2]''')

#replace substitui oq vc passa no primeiro argumento pelo segundo
numeroPaginas = numeroPaginas.text.replace("of ","")  

numeroPaginas = int(numeroPaginas)

print(numeroPaginas)

31


In [291]:
#lista vazia que sera usada para armazenar os dados da tabela por página
lista_de_tabela_por_pagina = []

#loop para percorrer cada uma das paginas
for pagina in range(0, numeroPaginas):

    #find element do driver do selenium encontra a tabela atual
    tabela = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/
                                    div/div/div/div[2]/section[2]/div[2]/div/table''')
                                    
    # obtém o atributo "outerHTML" da tabela encontrada e armazena em uma variável  "html_tabela". 
    # atributo "outerHTML" é o HTML completo da tabela, incluindo todos os seus elementos filhos.
    html_tabela = tabela.get_attribute("outerHTML")
    #passa p pandas p ficar legível
    tabela_final = pd.read_html(html_tabela)[0]
    #adiciona tabela final a lista de tabelas p pag
    lista_de_tabela_por_pagina.append(tabela_final)
    #acha o botão de avançar da pagina (p passar uma p uma)
    botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
    #clica nele
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)
    
#concatena todas as paginas em uma unica base de dados
base_dados_completa = pd.concat(lista_de_tabela_por_pagina)

display(base_dados_completa)



,Ticker,Name,Segment,Issuer,Expense Ratio,AUM
0,SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$384.58B
1,IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$313.53B
2,VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$287.40B
3,VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$285.32B
4,QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$164.12B
...,...,...,...,...,...,...
85,AWYX,ETFMG 2x Daily Travel Tech ETF,Leveraged Equity: Global Internet & Direct Mar...,ETFMG,0.95%,$341.97K
86,TADS,The Active Dividend Stock ETF,Equity: U.S. - Total Market,"Tuttle Tactical Management, LLC",1.68%,$294.89K
87,CRYP,AdvisorShares Managed Bitcoin Strategy ETF,Asset Allocation: Global Target Outcome,AdvisorShares,1.59%,$217.68K
88,FLRU,Franklin FTSE Russia ETF,Equity: Russia - Total Market,Franklin Templeton,0.19%,$8.00K


In [292]:
#primeira coisa é mudar a aba

botao_aba = driver.find_element("xpath", ''' /html/body/div[5]/section/div/div[3]/section/div/div/div/div/
div[2]/section[2]/div[2]/ul/li[2]/span''')

driver.execute_script("arguments[0].click();", botao_aba)

#voltando a 1 página

for pagina in range(0, numeroPaginas):
    
    botao_voltar_pagina = driver.find_element("xpath", '//*[@id="previousPage"]')
    
    driver.execute_script("arguments[0].click();", botao_voltar_pagina)

In [293]:
performace = []

for pagina in range(0, numeroPaginas):

    tabela = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/
                                    div/div/div/div[2]/section[2]/div[2]/div/table''')

    html_tabela = tabela.get_attribute("outerHTML")

    tabela_final = pd.read_html(html_tabela)[0]

    performace.append(tabela_final)
    
    botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
    
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)
    
#mesma coisa só muda o tipo de dados, ao inves do nome do etf vai p perfomace
base_de_dados_performance = pd.concat(performace)
display(base_de_dados_performance)



,Ticker,Name,1 Month,3 Month,YTD,1 Year,3 Years,5 Years,10 Years,As Of Date
0,SPY,SPDR S&P 500 ETF Trust,4.03%,8.84%,3.52%,-9.95%,7.75%,8.99%,12.39%,01/20/23
1,IVV,iShares Core S&P 500 ETF,4.06%,8.84%,3.51%,-9.95%,7.75%,9.03%,12.44%,01/20/23
2,VTI,Vanguard Total Stock Market ETF,4.49%,8.98%,3.96%,-10.36%,7.24%,8.49%,12.02%,01/20/23
3,VOO,Vanguard S&P 500 ETF,4.02%,8.82%,3.52%,-9.98%,7.73%,9.02%,12.44%,01/20/23
4,QQQ,Invesco QQQ Trust,4.88%,5.30%,6.16%,-21.28%,8.84%,11.98%,16.55%,01/20/23
...,...,...,...,...,...,...,...,...,...,...
85,AWYX,ETFMG 2x Daily Travel Tech ETF,36.11%,38.66%,31.27%,-48.46%,--,--,--,01/20/23
86,TADS,The Active Dividend Stock ETF,0%,0%,0%,0%,--,--,--,01/20/23
87,CRYP,AdvisorShares Managed Bitcoin Strategy ETF,--,--,--,--,--,--,--,01/20/23
88,FLRU,Franklin FTSE Russia ETF,0%,0%,0%,-66.37%,-31.18%,--,--,01/20/23


In [294]:
driver.quit()  #fecha navegador

In [295]:
#eliminando a coluna que mostra os numeros de indice, ou seja, deixando a primeira como ticker
base_dados_completa = base_dados_completa.set_index("Ticker")

base_dados_completa 


,Name,Segment,Issuer,Expense Ratio,AUM
Ticker,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$384.58B
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$313.53B
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$287.40B
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$285.32B
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$164.12B
...,...,...,...,...,...
AWYX,ETFMG 2x Daily Travel Tech ETF,Leveraged Equity: Global Internet & Direct Mar...,ETFMG,0.95%,$341.97K
TADS,The Active Dividend Stock ETF,Equity: U.S. - Total Market,"Tuttle Tactical Management, LLC",1.68%,$294.89K
CRYP,AdvisorShares Managed Bitcoin Strategy ETF,Asset Allocation: Global Target Outcome,AdvisorShares,1.59%,$217.68K


In [296]:
#eliminando a coluna que mostra os numeros de indice, ou seja, deixando a primeira como ticker
base_de_dados_performance = base_de_dados_performance.set_index("Ticker")

#pegando apenas colunas especificas
base_de_dados_performance = base_de_dados_performance[['1 Month', '1 Year', '3 Years', '5 Years', '10 Years']]

base_de_dados_performance

,1 Month,1 Year,3 Years,5 Years,10 Years
Ticker,,,,,
SPY,4.03%,-9.95%,7.75%,8.99%,12.39%
IVV,4.06%,-9.95%,7.75%,9.03%,12.44%
VTI,4.49%,-10.36%,7.24%,8.49%,12.02%
VOO,4.02%,-9.98%,7.73%,9.02%,12.44%
QQQ,4.88%,-21.28%,8.84%,11.98%,16.55%
...,...,...,...,...,...
AWYX,36.11%,-48.46%,--,--,--
TADS,0%,0%,--,--,--
CRYP,--,--,--,--,--


In [297]:
base_de_dados_performance = base_de_dados_performance[['3 Years', '5 Years', '10 Years']]

base_de_dados_performance

,3 Years,5 Years,10 Years
Ticker,,,
SPY,7.75%,8.99%,12.39%
IVV,7.75%,9.03%,12.44%
VTI,7.24%,8.49%,12.02%
VOO,7.73%,9.02%,12.44%
QQQ,8.84%,11.98%,16.55%
...,...,...,...
AWYX,--,--,--
TADS,--,--,--
CRYP,--,--,--


In [298]:
#join junta duas tabelas diferente a partir de uma chave, no caso 'ticker'
base_final = base_dados_completa.join(base_de_dados_performance)

display(base_final)

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$384.58B,7.75%,8.99%,12.39%
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$313.53B,7.75%,9.03%,12.44%
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$287.40B,7.24%,8.49%,12.02%
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$285.32B,7.73%,9.02%,12.44%
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$164.12B,8.84%,11.98%,16.55%
...,...,...,...,...,...,...,...,...
AWYX,ETFMG 2x Daily Travel Tech ETF,Leveraged Equity: Global Internet & Direct Mar...,ETFMG,0.95%,$341.97K,--,--,--
TADS,The Active Dividend Stock ETF,Equity: U.S. - Total Market,"Tuttle Tactical Management, LLC",1.68%,$294.89K,--,--,--
CRYP,AdvisorShares Managed Bitcoin Strategy ETF,Asset Allocation: Global Target Outcome,AdvisorShares,1.59%,$217.68K,--,--,--


# como analisar os dados de rentabilidade

## passo a passo

1. puxar as tabelas - ok
2. escolher qual período de rentabilidade analisar - 1 mes, 1 ano e 5 anos
3. retirar dados faltantes, fundo q n tem periodos ficarão de fora
4. transformar rentab. em numeros decimais inves de string
5. juntar tabelas
6. filtrar ETF alavancados (e excluí-los, rs.)
7. fazer ranking de rentabilidades p período e soma-los
8. ordenar do menor p maior, encontrando os "melhores"

In [299]:
#tirando dados faltantes
                                    #replace substitui os tracinhos por NaN
base_de_dados_performance = base_de_dados_performance.replace("--", pd.NA)
base_de_dados_performance

,3 Years,5 Years,10 Years
Ticker,,,
SPY,7.75%,8.99%,12.39%
IVV,7.75%,9.03%,12.44%
VTI,7.24%,8.49%,12.02%
VOO,7.73%,9.02%,12.44%
QQQ,8.84%,11.98%,16.55%
...,...,...,...
AWYX,<NA>,NaN,NaN
TADS,<NA>,NaN,NaN
CRYP,<NA>,NaN,NaN


In [300]:
#elimina os nan
base_de_dados_performance = base_de_dados_performance.dropna()
base_de_dados_performance

,3 Years,5 Years,10 Years
Ticker,,,
SPY,7.75%,8.99%,12.39%
IVV,7.75%,9.03%,12.44%
VTI,7.24%,8.49%,12.02%
VOO,7.73%,9.02%,12.44%
QQQ,8.84%,11.98%,16.55%
...,...,...,...
SZK,-26.41%,-20.50%,-22.95%
LD,2.50%,-5.95%,-1.99%
ERUS,-41.79%,-23.10%,-13.14%


In [301]:
    #para base de perfomarce de um mes = base na coluna 1 mes, 
    # strip é corte p tirar o simbolo de porcentagem, converte p float e divide por 100.

base_de_dados_performance['3 Years'] = (base_de_dados_performance["3 Years"].
                                        str.rstrip('%').astype(float)/100)

base_de_dados_performance['5 Years'] = (base_de_dados_performance["5 Years"].
                                        str.rstrip('%').astype(float)/100)

base_de_dados_performance['10 Years'] = (base_de_dados_performance["10 Years"].
                                        str.rstrip('%').astype(float)/100)



base_de_dados_performance

C:\Users\pulpp\AppData\Local\Temp\ipykernel_2464\1477903028.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_de_dados_performance['3 Years'] = (base_de_dados_performance["3 Years"].
C:\Users\pulpp\AppData\Local\Temp\ipykernel_2464\1477903028.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_de_dados_performance['5 Years'] = (base_de_dados_performance["5 Years"].
C:\Users\pulpp\AppData\Local\Temp\ipykernel_2464\1477903028.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,3 Years,5 Years,10 Years
Ticker,,,
SPY,0.0775,0.0899,0.1239
IVV,0.0775,0.0903,0.1244
VTI,0.0724,0.0849,0.1202
VOO,0.0773,0.0902,0.1244
QQQ,0.0884,0.1198,0.1655
...,...,...,...
SZK,-0.2641,-0.2050,-0.2295
LD,0.0250,-0.0595,-0.0199
ERUS,-0.4179,-0.2310,-0.1314


In [312]:
#how faz a interceção, ou seja, só os dados que as duas tabelas possuem
base_final = base_dados_completa.join(base_de_dados_performance, how = "inner")

base_final

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$384.58B,0.0775,0.0899,0.1239
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$313.53B,0.0775,0.0903,0.1244
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$287.40B,0.0724,0.0849,0.1202
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$285.32B,0.0773,0.0902,0.1244
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$164.12B,0.0884,0.1198,0.1655
...,...,...,...,...,...,...,...,...
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.28M,-0.2641,-0.2050,-0.2295
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.23M,0.0250,-0.0595,-0.0199
ERUS,iShares MSCI Russia ETF,Equity: Russia - Total Market,Blackrock,0.08%,$961.13K,-0.4179,-0.2310,-0.1314


In [313]:
#filtrando a coluna segmento que contém a palavra Leveraged (alavancado)

#quando coloca o tiuzin, você manda o sistema fazer o contrário
# NÃO pegue só os alavancados

base_final = base_final[~base_final['Segment'].str.contains("Leveraged")]
display(base_final)

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$384.58B,0.0775,0.0899,0.1239
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$313.53B,0.0775,0.0903,0.1244
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$287.40B,0.0724,0.0849,0.1202
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$285.32B,0.0773,0.0902,0.1244
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$164.12B,0.0884,0.1198,0.1655
...,...,...,...,...,...,...,...,...
RXD,ProShares UltraShort Health Care,Inverse Equity: U.S. Health Care,ProShares,0.95%,$1.92M,-0.2775,-0.2535,-0.2925
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.28M,-0.2641,-0.2050,-0.2295
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.23M,0.0250,-0.0595,-0.0199


In [314]:
#ascendente falso é decrescente, ou seja do maior p menor
#codigo rank, faz o python classificar

base_final['Rank 3 anos'] = base_final['3 Years'].rank(ascending = False)
base_final['Rank 5 anos'] = base_final['5 Years'].rank(ascending = False)
base_final['Rank 10 anos'] = base_final['10 Years'].rank(ascending = False)

base_final

C:\Users\pulpp\AppData\Local\Temp\ipykernel_2464\3976434740.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_final['Rank 3 anos'] = base_final['3 Years'].rank(ascending = False)
C:\Users\pulpp\AppData\Local\Temp\ipykernel_2464\3976434740.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_final['Rank 5 anos'] = base_final['5 Years'].rank(ascending = False)
C:\Users\pulpp\AppData\Local\Temp\ipykernel_2464\3976434740.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years,Rank 3 anos,Rank 5 anos,Rank 10 anos
Ticker,,,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$384.58B,0.0775,0.0899,0.1239,202.0,108.0,78.5
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$313.53B,0.0775,0.0903,0.1244,202.0,101.5,76.0
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$287.40B,0.0724,0.0849,0.1202,235.0,137.0,104.5
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$285.32B,0.0773,0.0902,0.1244,204.5,104.0,76.0
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$164.12B,0.0884,0.1198,0.1655,151.0,22.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...
RXD,ProShares UltraShort Health Care,Inverse Equity: U.S. Health Care,ProShares,0.95%,$1.92M,-0.2775,-0.2535,-0.2925,805.0,811.0,818.0
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.28M,-0.2641,-0.2050,-0.2295,804.0,801.0,807.0
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.23M,0.0250,-0.0595,-0.0199,451.0,760.0,703.5


In [322]:
base_final['rank_final'] = (
                            base_final['Rank 3 anos'] + 
                            base_final['Rank 5 anos'] + 
                            base_final['Rank 10 anos'] 
                            )


base_final


C:\Users\pulpp\AppData\Local\Temp\ipykernel_2464\3328011273.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_final['rank_final'] = (


,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years,Rank 3 anos,Rank 5 anos,Rank 10 anos,rank_final
Ticker,,,,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$384.58B,0.0775,0.0899,0.1239,202.0,108.0,78.5,388.5
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$313.53B,0.0775,0.0903,0.1244,202.0,101.5,76.0,379.5
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$287.40B,0.0724,0.0849,0.1202,235.0,137.0,104.5,476.5
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$285.32B,0.0773,0.0902,0.1244,204.5,104.0,76.0,384.5
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$164.12B,0.0884,0.1198,0.1655,151.0,22.0,13.0,186.0
...,...,...,...,...,...,...,...,...,...,...,...,...
RXD,ProShares UltraShort Health Care,Inverse Equity: U.S. Health Care,ProShares,0.95%,$1.92M,-0.2775,-0.2535,-0.2925,805.0,811.0,818.0,2434.0
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.28M,-0.2641,-0.2050,-0.2295,804.0,801.0,807.0,2412.0
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.23M,0.0250,-0.0595,-0.0199,451.0,760.0,703.5,1914.5


In [325]:
#sort_value classifica a tabela de acordo com o rank final
melhores = base_final.sort_values(by="rank_final")

melhores

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years,Rank 3 anos,Rank 5 anos,Rank 10 anos,rank_final
Ticker,,,,,,,,,,,,
TAN,Invesco Solar ETF,Equity: Global Renewable Energy,Invesco,0.69%,$2.53B,0.3308,0.2532,0.1735,2.0,1.0,8.0,11.0
QCLN,First Trust NASDAQ Clean Edge Green Energy Ind...,Equity: U.S. Energy,First Trust,0.58%,$1.87B,0.2418,0.1953,0.1860,13.0,3.0,5.0,21.0
XSD,SPDR S&P Semiconductor ETF,Equity: U.S. Semiconductors,State Street Global Advisors,0.35%,$1.35B,0.1900,0.2060,0.2364,29.5,2.0,1.0,32.5
SMH,VanEck Semiconductor ETF,Equity: Global Semiconductors,VanEck,0.35%,$7.95B,0.1677,0.1766,0.2242,37.0,5.0,4.0,46.0
PSI,Invesco Dynamic Semiconductors ETF,Equity: U.S. Semiconductors,Invesco,0.56%,$559.41M,0.1612,0.1541,0.2286,43.0,9.0,2.0,54.0
...,...,...,...,...,...,...,...,...,...,...,...,...
FAZ,Direxion Daily Financial Bear 3X Shares,Inverse Equity: U.S. Financials,Direxion,1.01%,$172.05M,-0.5603,-0.4670,-0.4593,832.0,832.0,831.0,2495.0
SSG,ProShares UltraShort Semiconductors,Inverse Equity: U.S. Semiconductors,ProShares,0.95%,$6.07M,-0.5494,-0.4862,-0.4707,831.0,833.0,833.0,2497.0
SQQQ,ProShares UltraPro Short QQQ,Inverse Equity: U.S. - Large Cap,ProShares,0.95%,$4.77B,-0.5451,-0.5159,-0.5103,830.0,834.0,834.0,2498.0


In [333]:
melhores = melhores[['Name', '3 Years',	'5 Years',	'10 Years',	'Rank 3 anos',	'Rank 5 anos',	'Rank 10 anos',	'rank_final']]

In [334]:
melhores = melhores.head(5)
melhores

,Name,3 Years,5 Years,10 Years,Rank 3 anos,Rank 5 anos,Rank 10 anos,rank_final
Ticker,,,,,,,,
TAN,Invesco Solar ETF,0.3308,0.2532,0.1735,2.0,1.0,8.0,11.0
QCLN,First Trust NASDAQ Clean Edge Green Energy Ind...,0.2418,0.1953,0.1860,13.0,3.0,5.0,21.0
XSD,SPDR S&P Semiconductor ETF,0.1900,0.2060,0.2364,29.5,2.0,1.0,32.5
SMH,VanEck Semiconductor ETF,0.1677,0.1766,0.2242,37.0,5.0,4.0,46.0
PSI,Invesco Dynamic Semiconductors ETF,0.1612,0.1541,0.2286,43.0,9.0,2.0,54.0
